In [81]:
'''
# NFL Analysis to-do's

- recreate fantasy point calculations for each position (Jar and Cam league) 
- prob create [temp] table for each position 
- analyze top teams each year for offensive output. Also analyze trend (maybe weekly trend and seasonal trend) 
- predict future fantasy points based on previous years 

'''

"\n# NFL Analysis to-do's\n\n- recreate fantasy point calculations for each position (Jar and Cam league) \n- prob create [temp] table for each position \n- analyze top teams each year for offensive output. Also analyze trend (maybe weekly trend and seasonal trend) \n- predict future fantasy points based on previous years \n\n"

In [82]:
# use this to insert one file into a table for sure 
# basically create a table and then load up one test file 
import psycopg2
import pandas as pd
import os


conn = psycopg2.connect(
    dbname='brdb',
    user='brandon',
    password='',
    host='localhost',
    port='5432'
)


## QB Stats

In [83]:
# Create cursor and execute query
query = "SELECT * FROM nfl_qb_data;"
df = pd.read_sql(query, conn)

# Close connection
#conn.close()

df

/var/folders/qb/19k1kp_s5m3bbj28vn082fl00000gp/T/ipykernel_8347/1798147076.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,player_name,player_display_name,recent_team,season,week,position,completions,attempts,passing_yards,passing_tds,interceptions,carries,rushing_yards,rushing_tds,fantasy_points,jar_pts,cam_pts
0,T.Brady,Tom Brady,TB,2022,1,QB,18.0,27.0,212.0,1.0,1.0,2.0,-1.0,0.0,10.38,10.38,12.38
1,T.Brady,Tom Brady,TB,2022,2,QB,18.0,34.0,190.0,1.0,0.0,3.0,-2.0,0.0,9.40,11.40,13.40
2,T.Brady,Tom Brady,TB,2022,3,QB,31.0,42.0,271.0,1.0,0.0,1.0,-1.0,0.0,14.74,14.74,16.74
3,T.Brady,Tom Brady,TB,2022,4,QB,39.0,52.0,385.0,3.0,0.0,0.0,0.0,0.0,25.40,27.40,33.40
4,T.Brady,Tom Brady,TB,2022,5,QB,35.0,52.0,351.0,1.0,0.0,3.0,-3.0,0.0,19.74,17.74,19.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970,C.Williams,Caleb Williams,CHI,2024,14,QB,17.0,23.0,134.0,2.0,0.0,4.0,27.0,0.0,14.06,16.06,20.06
1971,C.Williams,Caleb Williams,CHI,2024,15,QB,18.0,31.0,191.0,1.0,0.0,4.0,3.0,0.0,9.94,11.94,13.94
1972,C.Williams,Caleb Williams,CHI,2024,16,QB,26.0,40.0,334.0,2.0,0.0,6.0,34.0,0.0,22.76,24.76,28.76
1973,C.Williams,Caleb Williams,CHI,2024,17,QB,16.0,28.0,122.0,0.0,1.0,5.0,37.0,0.0,6.58,6.58,6.58


In [84]:
player_stats = df.groupby(['player_display_name','season']).agg({
            'week': ['count'],  # Number of games played
            'attempts': ['sum', 'mean', 'std', 'min', 'max'],
            'passing_yards': ['sum', 'mean', 'std', 'min', 'max'],
            'passing_tds': ['sum', 'mean', 'std', 'min', 'max'],
            'interceptions': ['sum', 'mean', 'std', 'min', 'max'],
            'rushing_yards': ['sum', 'mean', 'std', 'min', 'max'],
            'rushing_tds': ['sum', 'mean', 'std', 'min', 'max'],
            'jar_pts': ['count', 'sum', 'mean', 'std', 'min', 'max'], 
            'cam_pts': ['count', 'sum', 'mean', 'std', 'min', 'max']
        }).reset_index().round(2)


player_stats.columns = [
    '_'.join(col).strip('_') for col in player_stats.columns.values
]






In [85]:
# Filter by year-level counts - keep
summary = player_stats[player_stats['week_count'] > 4]


pivot_df = summary.pivot_table(
    index='player_display_name',
    columns='season',
    values=['week_count', 'passing_yards_sum', 'passing_tds_sum', 
            'rushing_yards_sum', 'rushing_tds_sum',
            'cam_pts_sum', 'cam_pts_mean', 'jar_pts_sum', 'jar_pts_mean']
)

# Flatten column MultiIndex
pivot_df.columns = [f'{metric}_{year}' for metric, year in pivot_df.columns]
qb_pivot = pivot_df.reset_index()

qb_pivot


,player_display_name,cam_pts_mean_2022,cam_pts_mean_2023,cam_pts_mean_2024,cam_pts_sum_2022,cam_pts_sum_2023,cam_pts_sum_2024,jar_pts_mean_2022,jar_pts_mean_2023,jar_pts_mean_2024,...,passing_yards_sum_2024,rushing_tds_sum_2022,rushing_tds_sum_2023,rushing_tds_sum_2024,rushing_yards_sum_2022,rushing_yards_sum_2023,rushing_yards_sum_2024,week_count_2022,week_count_2023,week_count_2024
0,Aaron Rodgers,17.36,NaN,18.39,295.20,NaN,312.58,14.31,NaN,15.09,...,3897.0,1.0,NaN,0.0,94.0,NaN,107.0,17.0,NaN,17.0
1,Aidan O'Connell,NaN,13.98,12.61,NaN,153.82,113.48,NaN,11.80,10.83,...,1612.0,NaN,1.0,1.0,NaN,11.0,30.0,NaN,11.0,9.0
2,Andy Dalton,15.02,NaN,12.16,210.24,NaN,72.96,12.45,NaN,9.83,...,989.0,0.0,NaN,0.0,54.0,NaN,34.0,14.0,NaN,6.0
3,Anthony Richardson,NaN,NaN,16.59,NaN,NaN,182.46,NaN,NaN,15.13,...,1814.0,NaN,NaN,6.0,NaN,NaN,499.0,NaN,NaN,11.0
4,Bailey Zappe,NaN,8.32,NaN,NaN,83.18,NaN,NaN,7.12,NaN,...,NaN,NaN,1.0,NaN,NaN,83.0,NaN,NaN,10.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,Tyler Huntley,10.40,6.32,14.13,52.02,31.62,70.66,9.60,5.12,12.93,...,829.0,1.0,0.0,2.0,137.0,55.0,135.0,5.0,5.0,5.0
77,Tyrod Taylor,NaN,9.73,NaN,NaN,97.34,NaN,NaN,8.73,NaN,...,NaN,NaN,0.0,NaN,NaN,197.0,NaN,NaN,10.0,NaN
78,Tyson Bagent,NaN,12.65,NaN,NaN,63.26,NaN,NaN,11.45,NaN,...,NaN,NaN,2.0,NaN,NaN,109.0,NaN,NaN,5.0,NaN
79,Will Levis,NaN,13.78,13.00,NaN,124.02,155.94,NaN,12.00,10.83,...,2091.0,NaN,1.0,0.0,NaN,57.0,183.0,NaN,9.0,12.0


## WR

In [86]:
# Create cursor and execute query
query = "SELECT * FROM nfl_wr_data;"
df = pd.read_sql(query, conn)

# Close connection
#conn.close()

df

/var/folders/qb/19k1kp_s5m3bbj28vn082fl00000gp/T/ipykernel_8347/2229702022.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,player_name,player_display_name,recent_team,season,week,position,carries,rushing_yards,rushing_tds,targets,receptions,receiving_yards,receiving_tds,fantasy_points,jar_pts,cam_pts
0,D.Jackson,DeSean Jackson,BAL,2022,9,WR,0.0,0.0,0.0,2.0,1.0,16.0,0.0,1.6,2.1,2.1
1,D.Jackson,DeSean Jackson,BAL,2022,12,WR,0.0,0.0,0.0,3.0,2.0,74.0,0.0,7.4,8.4,8.4
2,D.Jackson,DeSean Jackson,BAL,2022,13,WR,0.0,0.0,0.0,3.0,2.0,10.0,0.0,1.0,2.0,2.0
3,D.Jackson,DeSean Jackson,BAL,2022,14,WR,0.0,0.0,0.0,2.0,2.0,34.0,0.0,3.4,4.4,4.4
4,D.Jackson,DeSean Jackson,BAL,2022,15,WR,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6476,M.Corley,Malachi Corley,NYJ,2024,2,WR,0.0,0.0,0.0,1.0,1.0,4.0,0.0,0.4,0.9,0.9
6477,M.Corley,Malachi Corley,NYJ,2024,9,WR,1.0,18.0,0.0,0.0,0.0,0.0,0.0,1.8,1.8,1.8
6478,M.Corley,Malachi Corley,NYJ,2024,10,WR,0.0,0.0,0.0,2.0,1.0,2.0,0.0,0.2,0.7,0.7
6479,M.Corley,Malachi Corley,NYJ,2024,11,WR,0.0,0.0,0.0,1.0,1.0,10.0,0.0,1.0,1.5,1.5


In [87]:
player_stats = df.groupby(['player_display_name','season']).agg({
            'week': ['count'],  # Number of games played
            'targets': ['sum', 'mean', 'std', 'min', 'max'],
            'receptions': ['sum', 'mean', 'std', 'min', 'max'],
            'receiving_yards': ['sum', 'mean', 'std', 'min', 'max'],
            'receiving_tds': ['sum', 'mean', 'std', 'min', 'max'],
            'rushing_yards': ['sum', 'mean', 'std', 'min', 'max'],
            'rushing_tds': ['sum', 'mean', 'std', 'min', 'max'],
            'jar_pts': ['count', 'sum', 'mean', 'std', 'min', 'max'], 
            'cam_pts': ['count', 'sum', 'mean', 'std', 'min', 'max']
        }).reset_index().round(2)

# Flatten multi-level column names
#player_stats.columns = ['_'.join(col).strip('_') for col in player_stats.columns.values]

#player_stats.columns = ['_'.join(col).strip('_') for col in player_stats.columns.values]

player_stats.columns = [
    '_'.join(col).strip('_') for col in player_stats.columns.values
]



In [88]:
# Filter by year-level counts - keep
summary = player_stats[player_stats['week_count'] > 4]


pivot_df = summary.pivot_table(
    index='player_display_name',
    columns='season',
    values=['week_count', 'targets_sum', 'receptions_sum', 
            'receiving_yards_sum', 'receiving_tds_sum', 'rushing_yards_sum', 'rushing_tds_sum',
            'cam_pts_sum', 'cam_pts_mean', 'jar_pts_sum', 'jar_pts_mean']
)

# Flatten column MultiIndex
pivot_df.columns = [f'{metric}_{year}' for metric, year in pivot_df.columns]
wr_pivot = pivot_df.reset_index()



In [89]:
wr_pivot

,player_display_name,cam_pts_mean_2022,cam_pts_mean_2023,cam_pts_mean_2024,cam_pts_sum_2022,cam_pts_sum_2023,cam_pts_sum_2024,jar_pts_mean_2022,jar_pts_mean_2023,jar_pts_mean_2024,...,rushing_tds_sum_2024,rushing_yards_sum_2022,rushing_yards_sum_2023,rushing_yards_sum_2024,targets_sum_2022,targets_sum_2023,targets_sum_2024,week_count_2022,week_count_2023,week_count_2024
0,A.J. Brown,15.27,14.15,14.11,259.6,240.6,183.4,15.27,14.15,14.11,...,0.0,0.0,0.0,0.0,145.0,158.0,97.0,17.0,17.0,13.0
1,A.J. Green,3.66,NaN,NaN,47.6,NaN,NaN,3.66,NaN,NaN,...,NaN,0.0,NaN,NaN,47.0,NaN,NaN,13.0,NaN,NaN
2,A.T. Perry,NaN,6.82,NaN,NaN,54.6,NaN,NaN,6.82,NaN,...,NaN,NaN,0.0,NaN,NaN,18.0,NaN,NaN,8.0,NaN
3,Adam Thielen,8.41,10.44,11.55,143.0,177.5,115.5,8.41,10.44,11.55,...,0.0,4.0,6.0,0.0,107.0,137.0,62.0,17.0,17.0,10.0
4,Adonai Mitchell,NaN,NaN,2.55,NaN,NaN,43.3,NaN,NaN,2.55,...,0.0,NaN,NaN,6.0,NaN,NaN,55.0,NaN,NaN,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,Xavier Legette,NaN,NaN,6.71,NaN,NaN,100.6,NaN,NaN,6.71,...,0.0,NaN,NaN,24.0,NaN,NaN,84.0,NaN,NaN,15.0
255,Xavier Worthy,NaN,NaN,9.86,NaN,NaN,157.7,NaN,NaN,9.86,...,3.0,NaN,NaN,104.0,NaN,NaN,98.0,NaN,NaN,16.0
256,Zach Pascal,2.38,0.43,NaN,28.5,3.9,NaN,2.38,0.43,NaN,...,NaN,0.0,0.0,NaN,19.0,15.0,NaN,12.0,9.0,NaN
257,Zay Flowers,NaN,10.37,10.15,NaN,165.9,172.5,NaN,10.37,10.15,...,0.0,NaN,56.0,56.0,NaN,108.0,116.0,NaN,16.0,17.0


In [90]:
# RB 

In [91]:
# Create cursor and execute query
query = "SELECT * FROM nfl_rb_data;"
df = pd.read_sql(query, conn)

# Close connection
#conn.close()

df

/var/folders/qb/19k1kp_s5m3bbj28vn082fl00000gp/T/ipykernel_8347/68722915.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,player_name,player_display_name,recent_team,season,week,position,carries,rushing_yards,rushing_tds,targets,receptions,receiving_yards,receiving_tds,fantasy_points,jar_pts,cam_pts
0,M.Ingram,Mark Ingram,NO,2022,1,RB,4.0,22.0,0.0,1.0,1.0,1.0,0.0,0.3,2.8,2.8
1,M.Ingram,Mark Ingram,NO,2022,2,RB,10.0,58.0,0.0,2.0,2.0,3.0,0.0,4.1,7.1,7.1
2,M.Ingram,Mark Ingram,NO,2022,3,RB,5.0,18.0,1.0,0.0,0.0,0.0,0.0,7.8,7.8,7.8
3,M.Ingram,Mark Ingram,NO,2022,4,RB,10.0,30.0,0.0,3.0,3.0,13.0,0.0,4.3,5.8,5.8
4,M.Ingram,Mark Ingram,NO,2022,5,RB,9.0,16.0,0.0,0.0,0.0,0.0,0.0,3.6,1.6,1.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4066,T.Benson,Trey Benson,ARI,2024,10,RB,10.0,62.0,0.0,2.0,2.0,25.0,0.0,8.7,9.7,9.7
4067,T.Benson,Trey Benson,ARI,2024,12,RB,4.0,18.0,0.0,0.0,0.0,0.0,0.0,1.8,1.8,1.8
4068,T.Benson,Trey Benson,ARI,2024,13,RB,3.0,20.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0
4069,T.Benson,Trey Benson,ARI,2024,14,RB,2.0,15.0,0.0,1.0,1.0,4.0,0.0,1.9,2.4,2.4


In [92]:
player_stats = df.groupby(['player_display_name','season']).agg({
            'week': ['count'],  # Number of games played
            'targets': ['sum', 'mean', 'std', 'min', 'max'],
            'receptions': ['sum', 'mean', 'std', 'min', 'max'],
            'receiving_yards': ['sum', 'mean', 'std', 'min', 'max'],
            'receiving_tds': ['sum', 'mean', 'std', 'min', 'max'],
            'rushing_yards': ['sum', 'mean', 'std', 'min', 'max'],
            'rushing_tds': ['sum', 'mean', 'std', 'min', 'max'],
            'jar_pts': ['count', 'sum', 'mean', 'std', 'min', 'max'], 
            'cam_pts': ['count', 'sum', 'mean', 'std', 'min', 'max']
        }).reset_index().round(2)

# Flatten multi-level column names
#player_stats.columns = ['_'.join(col).strip('_') for col in player_stats.columns.values]

#player_stats.columns = ['_'.join(col).strip('_') for col in player_stats.columns.values]

player_stats.columns = [
    '_'.join(col).strip('_') for col in player_stats.columns.values
]




In [93]:
# Filter by year-level counts - keep
summary = player_stats[player_stats['week_count'] > 4]


pivot_df = summary.pivot_table(
    index='player_display_name',
    columns='season',
    values=['week_count', 'targets_sum', 'receptions_sum', 
            'receiving_yards_sum', 'receiving_tds_sum', 'rushing_yards_sum', 'rushing_tds_sum',
            'cam_pts_sum', 'cam_pts_mean', 'jar_pts_sum', 'jar_pts_mean']
)

# Flatten column MultiIndex
pivot_df.columns = [f'{metric}_{year}' for metric, year in pivot_df.columns]
rb_pivot = pivot_df.reset_index()



In [94]:
rb_pivot

,player_display_name,cam_pts_mean_2022,cam_pts_mean_2023,cam_pts_mean_2024,cam_pts_sum_2022,cam_pts_sum_2023,cam_pts_sum_2024,jar_pts_mean_2022,jar_pts_mean_2023,jar_pts_mean_2024,...,rushing_tds_sum_2024,rushing_yards_sum_2022,rushing_yards_sum_2023,rushing_yards_sum_2024,targets_sum_2022,targets_sum_2023,targets_sum_2024,week_count_2022,week_count_2023,week_count_2024
0,A.J. Dillon,9.04,NaN,NaN,153.6,NaN,NaN,9.04,NaN,NaN,...,NaN,770.0,NaN,NaN,43.0,NaN,NaN,17.0,NaN,NaN
1,AJ Dillon,NaN,7.11,NaN,NaN,106.6,NaN,NaN,7.11,NaN,...,NaN,NaN,613.0,NaN,NaN,28.0,NaN,NaN,15.0,NaN
2,Aaron Jones,13.12,11.08,13.06,223.1,121.9,222.1,13.12,11.08,13.06,...,5.0,1121.0,656.0,1138.0,72.0,43.0,62.0,17.0,11.0,17.0
3,Alexander Mattison,4.76,7.64,8.53,80.9,122.2,119.4,4.76,7.64,8.53,...,4.0,283.0,700.0,420.0,18.0,44.0,48.0,17.0,16.0,14.0
4,Alvin Kamara,12.75,14.58,16.52,191.2,189.5,231.3,12.75,14.58,16.52,...,6.0,897.0,694.0,950.0,77.0,86.0,89.0,15.0,13.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,Zach Charbonnet,NaN,5.60,9.57,NaN,89.6,162.7,NaN,5.60,9.57,...,8.0,NaN,462.0,569.0,NaN,40.0,52.0,NaN,16.0,17.0
159,Zack Moss,4.69,11.01,9.05,61.0,154.1,72.4,4.69,11.01,9.05,...,2.0,456.0,794.0,242.0,13.0,37.0,27.0,13.0,14.0,8.0
160,Zamir White,0.64,5.26,3.79,7.0,68.4,30.3,0.64,5.26,3.79,...,1.0,70.0,451.0,183.0,0.0,19.0,8.0,11.0,13.0,8.0
161,Zander Horvath,2.01,NaN,NaN,16.1,NaN,NaN,2.01,NaN,NaN,...,NaN,8.0,NaN,NaN,8.0,NaN,NaN,8.0,NaN,NaN


# TE 

In [95]:
# Create cursor and execute query
query = "SELECT * FROM nfl_te_data;"
df = pd.read_sql(query, conn)

# Close connection
#conn.close()

df

/var/folders/qb/19k1kp_s5m3bbj28vn082fl00000gp/T/ipykernel_8347/3173817822.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,player_name,player_display_name,recent_team,season,week,position,carries,rushing_yards,rushing_tds,targets,receptions,receiving_yards,receiving_tds,fantasy_points,jar_pts,cam_pts
0,M.Lewis,Marcedes Lewis,GB,2022,5,TE,0.0,0.0,0.0,1.0,1.0,2.0,1.0,6.2,6.7,6.7
1,M.Lewis,Marcedes Lewis,GB,2022,9,TE,0.0,0.0,0.0,1.0,1.0,19.0,0.0,1.9,2.4,2.4
2,M.Lewis,Marcedes Lewis,GB,2022,10,TE,0.0,0.0,0.0,1.0,1.0,-1.0,0.0,-0.1,0.4,0.4
3,M.Lewis,Marcedes Lewis,GB,2022,13,TE,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0
4,M.Lewis,Marcedes Lewis,GB,2022,15,TE,0.0,0.0,0.0,1.0,1.0,14.0,0.0,1.4,1.9,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3241,T.Johnson,Theo Johnson,NYG,2024,13,TE,0.0,0.0,0.0,5.0,5.0,54.0,0.0,5.4,7.9,7.9
3242,B.Sinnott,Ben Sinnott,WAS,2024,7,TE,0.0,0.0,0.0,2.0,2.0,6.0,1.0,6.6,7.6,7.6
3243,B.Sinnott,Ben Sinnott,WAS,2024,9,TE,0.0,0.0,0.0,1.0,1.0,12.0,0.0,1.2,1.7,1.7
3244,B.Sinnott,Ben Sinnott,WAS,2024,13,TE,0.0,0.0,0.0,1.0,1.0,3.0,0.0,0.3,0.8,0.8


In [96]:
player_stats = df.groupby(['player_display_name','season']).agg({
            'week': ['count'],  # Number of games played
            'targets': ['sum', 'mean', 'std', 'min', 'max'],
            'receptions': ['sum', 'mean', 'std', 'min', 'max'],
            'receiving_yards': ['sum', 'mean', 'std', 'min', 'max'],
            'receiving_tds': ['sum', 'mean', 'std', 'min', 'max'],
            'rushing_yards': ['sum', 'mean', 'std', 'min', 'max'],
            'rushing_tds': ['sum', 'mean', 'std', 'min', 'max'],
            'jar_pts': ['count', 'sum', 'mean', 'std', 'min', 'max'], 
            'cam_pts': ['count', 'sum', 'mean', 'std', 'min', 'max']
        }).reset_index().round(2)

# Flatten multi-level column names
#player_stats.columns = ['_'.join(col).strip('_') for col in player_stats.columns.values]

#player_stats.columns = ['_'.join(col).strip('_') for col in player_stats.columns.values]

player_stats.columns = [
    '_'.join(col).strip('_') for col in player_stats.columns.values
]




In [97]:
# Filter by year-level counts - keep
summary = player_stats[player_stats['week_count'] > 4]


pivot_df = summary.pivot_table(
    index='player_display_name',
    columns='season',
    values=['week_count', 'targets_sum', 'receptions_sum', 
            'receiving_yards_sum', 'receiving_tds_sum', 'rushing_yards_sum', 'rushing_tds_sum',
            'cam_pts_sum', 'cam_pts_mean', 'jar_pts_sum', 'jar_pts_mean']
)

# Flatten column MultiIndex
pivot_df.columns = [f'{metric}_{year}' for metric, year in pivot_df.columns]
te_pivot = pivot_df.reset_index()



In [98]:
te_pivot

,player_display_name,cam_pts_mean_2022,cam_pts_mean_2023,cam_pts_mean_2024,cam_pts_sum_2022,cam_pts_sum_2023,cam_pts_sum_2024,jar_pts_mean_2022,jar_pts_mean_2023,jar_pts_mean_2024,...,rushing_tds_sum_2024,rushing_yards_sum_2022,rushing_yards_sum_2023,rushing_yards_sum_2024,targets_sum_2022,targets_sum_2023,targets_sum_2024,week_count_2022,week_count_2023,week_count_2024
0,A.J. Barner,NaN,NaN,4.54,NaN,NaN,63.5,NaN,NaN,4.54,...,0.0,NaN,NaN,0.0,NaN,NaN,38.0,NaN,NaN,14.0
1,Adam Trautman,2.98,3.09,2.87,35.7,49.4,37.3,2.98,3.09,2.87,...,0.0,0.0,0.0,0.0,22.0,35.0,22.0,12.0,16.0,13.0
2,Albert Okwuegbunam,3.42,NaN,NaN,20.5,NaN,NaN,3.42,NaN,NaN,...,NaN,0.0,NaN,NaN,18.0,NaN,NaN,6.0,NaN,NaN
3,Andrew Ogletree,NaN,3.12,1.95,NaN,31.2,21.4,NaN,3.12,1.95,...,0.0,NaN,0.0,0.0,NaN,21.0,14.0,NaN,10.0,11.0
4,Anthony Firkser,1.81,NaN,NaN,14.5,NaN,NaN,1.81,NaN,NaN,...,NaN,0.0,NaN,NaN,13.0,NaN,NaN,8.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,Tyler Higbee,7.25,5.67,NaN,116.0,85.0,NaN,7.25,5.67,NaN,...,NaN,0.0,0.0,NaN,108.0,70.0,NaN,16.0,15.0,NaN
130,Will Dissly,4.66,2.64,5.67,69.9,31.7,85.1,4.66,2.64,5.67,...,0.0,0.0,0.0,0.0,38.0,22.0,64.0,15.0,12.0,15.0
131,Will Mallory,NaN,2.70,0.70,NaN,29.7,4.9,NaN,2.70,0.70,...,0.0,NaN,0.0,0.0,NaN,26.0,8.0,NaN,11.0,7.0
132,Zach Ertz,8.81,5.46,8.26,88.1,38.2,140.4,8.81,5.46,8.26,...,0.0,0.0,0.0,0.0,69.0,43.0,91.0,10.0,7.0,17.0
